In [153]:
pip install geopandas

In [154]:
import pandas as pd
import numpy as np
import math
import geopandas
import json

In [155]:
geo = geopandas.read_file('https://raw.githubusercontent.com/martinjc/UK-GeoJSON/master/json/administrative/eng/lad.json')

In [156]:
geo = geo.rename(columns={"LAD13NM": "Region Name", "LAD13CD": "Area Code" })
geo

,Area Code,LAD13CDO,Region Name,LAD13NMW,geometry
0,E06000001,00EB,Hartlepool,None,"MULTIPOLYGON (((-1.24196 54.72245, -1.24251 54..."
1,E06000002,00EC,Middlesbrough,None,"MULTIPOLYGON (((-1.19705 54.58178, -1.19436 54..."
2,E06000003,00EE,Redcar and Cleveland,None,"MULTIPOLYGON (((-1.20099 54.57759, -1.20024 54..."
3,E06000004,00EF,Stockton-on-Tees,None,"MULTIPOLYGON (((-1.30387 54.56514, -1.30199 54..."
4,E06000005,00EH,Darlington,None,"POLYGON ((-1.63799 54.61719, -1.63766 54.61669..."
...,...,...,...,...,...
321,E09000029,00BF,Sutton,None,"POLYGON ((-0.16646 51.38969, -0.16529 51.38798..."
322,E09000030,00BG,Tower Hamlets,None,"POLYGON ((-0.02947 51.54303, -0.02901 51.54227..."
323,E09000031,00BH,Waltham Forest,None,"POLYGON ((-0.00822 51.64361, -0.00668 51.64333..."
324,E09000032,00BJ,Wandsworth,None,"POLYGON ((-0.12636 51.48447, -0.12750 51.48202..."


In [157]:
prices_data = pd.read_csv('http://publicdata.landregistry.gov.uk/market-trend-data/house-price-index-data/Average-prices-2021-10.csv?utm_medium=GOV.UK&utm_source=datadownload&utm_campaign=average_price&utm_term=9.30_15_12_21')

In [158]:
prices_data = prices_data[prices_data['Date'].str.contains("2021-10-01")]

prices_data = prices_data.rename(columns={"Region_Name": "Region Name", "Area_Code": "Area Code" })
prices_data = prices_data.drop(['Monthly_Change', 'Annual_Change', 'Average_Price_SA','Average_Price_SA', 'Date'], axis=1)
prices_data['Average_Price'] = prices_data['Average_Price'].astype(float)
prices_data['Average_Price'] = prices_data['Average_Price'].apply(np.ceil) 
prices_data

,Region Name,Area Code,Average_Price
133566,England,E92000001,285114.0
133567,Northern Ireland,N92000001,159109.0
133568,Wales,W92000004,203224.0
133569,Scotland,S92000003,181392.0
133570,Inner London,E13000001,602501.0
...,...,...,...
133982,South Gloucestershire,E06000025,326397.0
133983,City of Plymouth,E06000026,204631.0
133984,England and Wales,K04000001,280701.0
133985,Great Britain,K03000001,271512.0


In [159]:
merged = pd.merge(geo, prices_data, on='Area Code', how='left')
merged = merged.drop(['LAD13NMW', 'LAD13CDO', 'Area Code', 'Region Name_y'], axis=1)
merged = merged.rename(columns={"Region Name_y": "Region Name", "Average_Price": "Average Price (£)", "Region Name_x": "Region Name",})
merged

,Region Name,geometry,Average Price (£)
0,Hartlepool,"MULTIPOLYGON (((-1.24196 54.72245, -1.24251 54...",134659.0
1,Middlesbrough,"MULTIPOLYGON (((-1.19705 54.58178, -1.19436 54...",132815.0
2,Redcar and Cleveland,"MULTIPOLYGON (((-1.20099 54.57759, -1.20024 54...",141096.0
3,Stockton-on-Tees,"MULTIPOLYGON (((-1.30387 54.56514, -1.30199 54...",148789.0
4,Darlington,"POLYGON ((-1.63799 54.61719, -1.63766 54.61669...",146919.0
...,...,...,...
321,Sutton,"POLYGON ((-0.16646 51.38969, -0.16529 51.38798...",417545.0
322,Tower Hamlets,"POLYGON ((-0.02947 51.54303, -0.02901 51.54227...",417722.0
323,Waltham Forest,"POLYGON ((-0.00822 51.64361, -0.00668 51.64333...",508312.0
324,Wandsworth,"POLYGON ((-0.12636 51.48447, -0.12750 51.48202...",634297.0


In [160]:
merged.to_file("HousePrices.geojson")  